<a href="https://colab.research.google.com/github/SkinHeadDressSom/Ultimate-X-ray-AI/blob/main/CXR_split_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.1 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
import pydicom
from PIL import Image
import cv2
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool

## making Yolo format

In [7]:
csv_path = "/content/drive/MyDrive/UltimateXray/ultimateXray.csv"
output_dir = "/content/drive/MyDrive/UltimateXray/preprocess"

image_output_dir = os.path.join(output_dir, "images")
label_output_dir = os.path.join(output_dir, "labels")

os.makedirs(image_output_dir, exist_ok=True)
os.makedirs(label_output_dir, exist_ok=True)

In [12]:
df = pd.read_csv(csv_path)
df = df[df['class_name']!='No Finding']
df

,image_id,class_name,x_center,y_center,width,height,image_path
45,00000032_037.png,Infiltrate,339.166138,119.195767,172.292063,351.085714,/content/drive/MyDrive/UltimateXray/NIH/train/...
114,00000072_000.png,Atelectasis,356.503704,567.805291,127.864550,82.353439,/content/drive/MyDrive/UltimateXray/NIH/train/...
222,00000147_001.png,Atelectasis,617.650794,599.229630,100.774603,179.877249,/content/drive/MyDrive/UltimateXray/NIH/train/...
229,00000149_006.png,Atelectasis,603.661017,460.239556,249.491525,90.033898,/content/drive/MyDrive/UltimateXray/NIH/train/...
233,00000150_002.png,Pneumonia,601.396825,595.978836,215.635979,180.960847,/content/drive/MyDrive/UltimateXray/NIH/train/...
...,...,...,...,...,...,...,...
138803,b53d1dd80e99ca6bcef9d592f65d3321,Pleural effusion,401.000000,1775.500000,322.000000,451.000000,/content/drive/MyDrive/UltimateXray/VinBigData...
138806,26d1d5a0ef2e692c6340e74859ffdc53,Pulmonary fibrosis,1250.500000,864.000000,175.000000,154.000000,/content/drive/MyDrive/UltimateXray/VinBigData...
138807,22672ab82c290c20b86863291e25ef6c,ILD,546.500000,1086.000000,495.000000,844.000000,/content/drive/MyDrive/UltimateXray/VinBigData...
138808,db169d0be36123bd55b866d6aa73983b,Other lesion,139.000000,1203.000000,266.000000,1066.000000,/content/drive/MyDrive/UltimateXray/VinBigData...


In [13]:
class_mapping = {name: idx for idx, name in enumerate(df['class_name'].unique())}
df['class_id'] = df['class_name'].map(class_mapping)
df

,image_id,class_name,x_center,y_center,width,height,image_path,class_id
45,00000032_037.png,Infiltrate,339.166138,119.195767,172.292063,351.085714,/content/drive/MyDrive/UltimateXray/NIH/train/...,0
114,00000072_000.png,Atelectasis,356.503704,567.805291,127.864550,82.353439,/content/drive/MyDrive/UltimateXray/NIH/train/...,1
222,00000147_001.png,Atelectasis,617.650794,599.229630,100.774603,179.877249,/content/drive/MyDrive/UltimateXray/NIH/train/...,1
229,00000149_006.png,Atelectasis,603.661017,460.239556,249.491525,90.033898,/content/drive/MyDrive/UltimateXray/NIH/train/...,1
233,00000150_002.png,Pneumonia,601.396825,595.978836,215.635979,180.960847,/content/drive/MyDrive/UltimateXray/NIH/train/...,2
...,...,...,...,...,...,...,...,...
138803,b53d1dd80e99ca6bcef9d592f65d3321,Pleural effusion,401.000000,1775.500000,322.000000,451.000000,/content/drive/MyDrive/UltimateXray/VinBigData...,16
138806,26d1d5a0ef2e692c6340e74859ffdc53,Pulmonary fibrosis,1250.500000,864.000000,175.000000,154.000000,/content/drive/MyDrive/UltimateXray/VinBigData...,12
138807,22672ab82c290c20b86863291e25ef6c,ILD,546.500000,1086.000000,495.000000,844.000000,/content/drive/MyDrive/UltimateXray/VinBigData...,10
138808,db169d0be36123bd55b866d6aa73983b,Other lesion,139.000000,1203.000000,266.000000,1066.000000,/content/drive/MyDrive/UltimateXray/VinBigData...,14


In [14]:
df['file_exists'] = df['image_path'].apply(lambda x: os.path.exists(x))
df = df[df['file_exists']].drop(columns=['file_exists'])
df

,image_id,class_name,x_center,y_center,width,height,image_path,class_id
45,00000032_037.png,Infiltrate,339.166138,119.195767,172.292063,351.085714,/content/drive/MyDrive/UltimateXray/NIH/train/...,0
114,00000072_000.png,Atelectasis,356.503704,567.805291,127.864550,82.353439,/content/drive/MyDrive/UltimateXray/NIH/train/...,1
222,00000147_001.png,Atelectasis,617.650794,599.229630,100.774603,179.877249,/content/drive/MyDrive/UltimateXray/NIH/train/...,1
229,00000149_006.png,Atelectasis,603.661017,460.239556,249.491525,90.033898,/content/drive/MyDrive/UltimateXray/NIH/train/...,1
233,00000150_002.png,Pneumonia,601.396825,595.978836,215.635979,180.960847,/content/drive/MyDrive/UltimateXray/NIH/train/...,2
...,...,...,...,...,...,...,...,...
138803,b53d1dd80e99ca6bcef9d592f65d3321,Pleural effusion,401.000000,1775.500000,322.000000,451.000000,/content/drive/MyDrive/UltimateXray/VinBigData...,16
138806,26d1d5a0ef2e692c6340e74859ffdc53,Pulmonary fibrosis,1250.500000,864.000000,175.000000,154.000000,/content/drive/MyDrive/UltimateXray/VinBigData...,12
138807,22672ab82c290c20b86863291e25ef6c,ILD,546.500000,1086.000000,495.000000,844.000000,/content/drive/MyDrive/UltimateXray/VinBigData...,10
138808,db169d0be36123bd55b866d6aa73983b,Other lesion,139.000000,1203.000000,266.000000,1066.000000,/content/drive/MyDrive/UltimateXray/VinBigData...,14


In [15]:
def convert_dicom_to_png_and_resize(dicom_path, png_path):
    dicom = pydicom.dcmread(dicom_path)
    image = dicom.pixel_array
    scaled_image = ((image - np.min(image)) / (np.max(image) - np.min(image)) * 255).astype(np.uint8)
    resized_image = Image.fromarray(scaled_image).resize((640, 640))
    resized_image.save(png_path)

In [16]:
def process_row(row):
    file_path = row['image_path']

    image_name = os.path.splitext(os.path.basename(file_path))[0]
    image_save_path = os.path.join(image_output_dir, f"{image_name}.png")
    label_save_path = os.path.join(label_output_dir, f"{image_name}.txt")

    if file_path.endswith('.png'):
        image = cv2.imread(file_path)
        if image is None:
            return
        image_height, image_width = image.shape[:2]
    else:
        dcm = pydicom.dcmread(file_path)
        if 'PixelData' not in dcm:
            return
        image = dcm.pixel_array
        image_height, image_width = image.shape

    if not os.path.exists(image_save_path):
        if file_path.endswith('.png'):
            img_resized = cv2.resize(image, (640, 640))
            cv2.imwrite(image_save_path, img_resized)
        else:
            image = Image.fromarray(image)
            img_resized = image.resize((640, 640))
            img_resized.save(image_save_path, "PNG")

    if row['x_center']:
        width = row['width'] / image_width
        height = row['height'] / image_height
        x_center = row['x_center'] / image_width
        y_center = row['y_center'] / image_height

        label = f"{row['class_id']} {x_center} {y_center} {width} {height}\n"
        with open(label_save_path, 'a') as f:
            f.write(label)

In [17]:
def parallel_process(df):
    args = [row for _, row in df.iterrows()]
    with Pool() as pool:
        list(tqdm(pool.imap(process_row, args), total=len(args)))

In [18]:
parallel_process(df)

100%|██████████| 44779/44779 [1:59:31<00:00,  6.24it/s]


In [19]:
print(len(os.listdir(os.path.join(image_output_dir))))
print(len(os.listdir(os.path.join(label_output_dir))))

9911
9911


## train val test split

In [20]:
import os
import random
import shutil
from pathlib import Path

In [21]:
images_dir = "/content/drive/MyDrive/UltimateXray/preprocess/images"
labels_dir = "/content/drive/MyDrive/UltimateXray/preprocess/labels"
base_dir = "/content/drive/MyDrive/UltimateXray/YOLO"

subdirs = {
    "train": {"images": f"{base_dir}/images/train", "labels": f"{base_dir}/labels/train"},
    "val": {"images": f"{base_dir}/images/val", "labels": f"{base_dir}/labels/val"},
    "test": {"images": f"{base_dir}/images/test", "labels": f"{base_dir}/labels/test"},
}

for split, paths in subdirs.items():
    os.makedirs(paths["images"], exist_ok=True)
    os.makedirs(paths["labels"], exist_ok=True)

In [22]:
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.png')])

labeled_images = []
unlabeled_images = []

In [23]:
for image in image_files:
    label_path = os.path.join(labels_dir, Path(image).stem + ".txt")
    if os.path.exists(label_path):
        labeled_images.append(image)
    else:
        unlabeled_images.append(image)

In [24]:
random.seed(42)
random.shuffle(labeled_images)
random.shuffle(unlabeled_images)

In [25]:
def split_data(data, train_ratio, val_ratio):
    train_size = int(len(data) * train_ratio)
    val_size = int(len(data) * val_ratio)
    train = data[:train_size]
    val = data[train_size:train_size + val_size]
    test = data[train_size + val_size:]
    return train, val, test

train_labeled, val_labeled, test_labeled = split_data(labeled_images, train_ratio, val_ratio)
train_unlabeled, val_unlabeled, test_unlabeled = split_data(unlabeled_images, train_ratio, val_ratio)

In [26]:
train_set = train_labeled + train_unlabeled
val_set = val_labeled + val_unlabeled
test_set = test_labeled + test_unlabeled

In [27]:
def copy_files(split, images, subdir):
    for image in tqdm(images):
        label_path = os.path.join(labels_dir, Path(image).stem + ".txt")
        image_src = os.path.join(images_dir, image)
        label_dst_dir = subdirs[subdir]["labels"]
        image_dst_dir = subdirs[subdir]["images"]

        # Copy image
        shutil.copy(image_src, image_dst_dir)

        # Copy label if it exists
        if os.path.exists(label_path):
            shutil.copy(label_path, label_dst_dir)

In [28]:
copy_files("train", train_set, "train")
copy_files("val", val_set, "val")
copy_files("test", test_set, "test")

100%|██████████| 992/992 [04:11<00:00,  3.95it/s]


In [29]:
class_names = df['class_name'].unique()
class_dict = {idx: name for idx, name in enumerate(class_names)}
class_dict

{0: 'Infiltrate',
 1: 'Atelectasis',
 2: 'Pneumonia',
 3: 'Cardiomegaly',
 4: 'Effusion',
 5: 'Pneumothorax',
 6: 'Mass',
 7: 'Nodule',
 8: 'Aortic enlargement',
 9: 'Pleural thickening',
 10: 'ILD',
 11: 'Nodule/Mass',
 12: 'Pulmonary fibrosis',
 13: 'Lung Opacity',
 14: 'Other lesion',
 15: 'Infiltration',
 16: 'Pleural effusion',
 17: 'Calcification',
 18: 'Consolidation'}

In [30]:
import yaml

base_path = "/content/drive/MyDrive/UltimateXray/YOLO"

dataset_config = {
    'path': base_dir,
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'names': class_dict
}

yaml_file_path = os.path.join(base_dir, 'dataset.yaml')

with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(dataset_config, yaml_file, default_flow_style=False)